In [2]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import kruskal, mannwhitneyu
import matplotlib.pyplot as plt
import os
import json
import re
from tabulate import tabulate


In [3]:

# 配置路径
base_dir = os.getcwd()
path = base_dir + '/../data/'

X_path = path + "package_substitutability.csv"
Y_path = path + "maintenance_activity.csv"
Z_path = path + "structure_criticality.csv"
directory_path = path + "/advisory-db-osv/crates/"


In [7]:

df_X = pd.read_csv(X_path).rename(columns={"package_name": "crate_name", "avg_top20_similarity": "X"})
df_Y = pd.read_csv(Y_path).rename(columns={"predicted_score": "Y"})
df_Z = pd.read_csv(Z_path).rename(columns={"PR": "Z"})

df = df_Y.merge(df_X, on="crate_name", how="left").merge(df_Z[["crate_name", "Z"]], on="crate_name", how="left")
df = df.dropna(subset=["X", "Y", "Z"])

# 使用标准化
scaler = StandardScaler()
df[["X", "Y", "Z"]] = scaler.fit_transform(df[["X", "Y", "Z"]])
alpha = 0.5
beta = 0.4
gama = 0.1
df["final_score"] = alpha * df["Z"] + beta * (1 - df["X"]) + gama * (1 - df["Y"])
df = df.rename(columns={'X': "可替代性", 'Y': "维护质量", 'Z': "结构关键性"})
sorted_df = df.sort_values(by="final_score", ascending=False).reset_index(drop=True)
sorted_df["rank"] = sorted_df.index + 1
print(sorted_df.head(20))


     crate_name  crate_id      维护质量      可替代性       结构关键性  final_score  rank
0   proc-macro2     22124  0.724761  1.320340  136.556847    68.177811     1
1         quote      6224 -0.514472  2.310418   68.154445    33.704502     2
2           syn      6274 -0.251605  2.872254   51.809603    25.281060     3
3     thiserror    171267  1.776232  2.019776   39.901687    19.465310     4
4        anyhow    170198  0.649656  2.200216   25.923403    12.516650     5
5     once_cell     77028  0.461894  0.217739   22.386490    11.559960     6
6         bytes      1315 -0.176500  1.078609   22.623129    11.397771     7
7        chrono       120  0.161473  1.971370   22.746283    11.068446     8
8        memchr      2364  1.550917  0.390459   15.006573     7.692011     9
9      bitflags       793 -0.176500  2.043621   14.448555     6.924479    10
10     textwrap      7649  1.813785  0.021823   10.238746     5.429266    11
11          url       109  1.888890  0.551179    9.350644     4.765962    12

In [ ]:
indicators_df = sorted_df[["可替代性", "维护质量", "结构关键性"]]
corr_matrix = indicators_df.corr(method="pearson")
print("三项指标之间的皮尔逊相关性矩阵：")
print(corr_matrix)


三项指标之间的皮尔逊相关性矩阵：
           可替代性      维护质量     结构关键性
可替代性   1.000000  0.018376  0.031460
维护质量   0.018376  1.000000  0.008894
结构关键性  0.031460  0.008894  1.000000


In [5]:

def parse_cvss_vector(vector_str):
    try:
        match = re.match(r"CVSS:\d+\.\d+/.*", vector_str)
        if match:
            from cvss import CVSS3
            return CVSS3(vector_str).base_score
    except:
        return None
    return None

def categorize_severity(score):
    if score is None:
        return "Unknown"
    if score < 4.0:
        return "Low"
    elif score < 7.0:
        return "Medium"
    elif score < 9.0:
        return "High"
    else:
        return "Critical"

cvss_scores = []
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        try:
            with open(os.path.join(directory_path, filename), "r", encoding="utf-8") as f:
                data = json.load(f)
                for item in data.get("affected", []):
                    name = item.get("package", {}).get("name")
                    if not name:
                        continue
                    max_score = None
                    for s in data.get("severity", []):
                        if s.get("type", "").startswith("CVSS"):
                            score = parse_cvss_vector(s.get("score", ""))
                            if score is not None:
                                max_score = max(score, max_score) if max_score else score
                    if max_score is not None:
                        cvss_scores.append({"crate_name": name, "cvss_score": max_score})
        except:
            continue
cvss_df = pd.DataFrame(cvss_scores)
cvss_df = cvss_df.groupby("crate_name", as_index=False)["cvss_score"].max()
cvss_df["severity"] = cvss_df["cvss_score"].apply(categorize_severity)
print(cvss_df)


        crate_name cvss_score  severity
0       abi_stable        7.5      High
1             abox        8.1      High
2      actix-codec        9.8  Critical
3       actix-http        7.5      High
4    actix-service        5.5    Medium
..             ...        ...       ...
242             ws        7.5      High
243            xcb        5.5    Medium
244      yaml-rust        7.5      High
245        yottadb        9.8  Critical
246        zlib-rs        5.3    Medium

[247 rows x 3 columns]


In [ ]:

cvss_rank_df = pd.merge(cvss_df, sorted_df[["crate_name", "rank", 'final_score']], on="crate_name")
cvss_rank_df["severity"] = cvss_rank_df["cvss_score"].apply(categorize_severity)

avg_rank_by_severity = (
    cvss_rank_df
    .groupby("severity")
    .agg(avg=("rank", "mean"), count=("rank", "count"))
    .reset_index()
    .sort_values(by="avg")
)
print("各 CVSS 严重性等级的风险评估平均排名及样本数量：")
print(avg_rank_by_severity)


各 CVSS 严重性等级的平均排名及样本数量：
   severity           avg  count
0  Critical  12368.428571     21
1      High  17521.972222     36
2    Medium  21411.350000     20


In [7]:

def run_one_sided_test(group1_label, group2_label, df, field="rank", direction="less"):
    group1 = df[df["severity"] == group1_label][field].values
    group2 = df[df["severity"] == group2_label][field].values

    if len(group1) < 3 or len(group2) < 3:
        print(f"⚠️ 样本不足：{group1_label} vs {group2_label}")
        return

    stat, p_value = mannwhitneyu(group1, group2, alternative=direction)
    print(f"📊 Mann–Whitney 单边检验：{group1_label} 的 {field} 是否 {direction} than {group2_label}")
    print(f"U 统计量 = {stat:.4f}, p 值 = {p_value:.4f}")
    if p_value < 0.05:
        print("✅ 差异具有统计显著性（p < 0.05）")
    else:
        print("❌ 差异不显著")

run_one_sided_test("Critical", "High", cvss_rank_df, "rank", "less")
run_one_sided_test("High", "Medium", cvss_rank_df, "rank", "less")
run_one_sided_test("Critical", "High", cvss_rank_df, "final_score", "greater")
run_one_sided_test("High", "Medium", cvss_rank_df, "final_score", "greater")


📊 Mann–Whitney 单边检验：Critical 的 rank 是否 less than High
U 统计量 = 271.0000, p 值 = 0.0390
✅ 差异具有统计显著性（p < 0.05）
📊 Mann–Whitney 单边检验：High 的 rank 是否 less than Medium
U 统计量 = 261.0000, p 值 = 0.0461
✅ 差异具有统计显著性（p < 0.05）
📊 Mann–Whitney 单边检验：Critical 的 final_score 是否 greater than High
U 统计量 = 485.0000, p 值 = 0.0390
✅ 差异具有统计显著性（p < 0.05）
📊 Mann–Whitney 单边检验：High 的 final_score 是否 greater than Medium
U 统计量 = 459.0000, p 值 = 0.0461
✅ 差异具有统计显著性（p < 0.05）
